In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AMAZONREVIEWS").getOrCreate()

In [4]:
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_DVD_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [ ]:
df.count()

5069140

In [10]:
cleaned_df=df.dropna()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [ ]:
parent_df=cleaned_df.select(["review_id", "customer_id", "product_id","product_parent","review_date"])
parent_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R33UPQQUZQEM8|   27288431|B005T4ND06|     400024643| 2015-08-31|
|R3IKTNQQPD9662|   13722556|B004EPZ070|     685335564| 2015-08-31|
|R3U27V5QMCP27T|   20381037|B005S9EKCW|     922008804| 2015-08-31|
|R2TOH2QKNK4IOC|   24852644|B00FC1ZCB4|     326560548| 2015-08-31|
|R2XQG5NJ59UFMY|   15556113|B002ZG98Z0|     637495038| 2015-08-31|
|R1N1KHBRR4ZTX3|    6132474|B00X8RONBO|     896602391| 2015-08-31|
|R3OM9S0TCBP38K|   48049524|B000CEXFZG|     115883890| 2015-08-31|
|R1W4S949ZRCTBW|    3282516|B00ID8H8EW|     977932459| 2015-08-31|
|R18JL1NNQAZFV2|   51771179|B000TGJ8IU|     840084782| 2015-08-31|
|R1LP6PR06OPYUX|   31816501|B00DPMPTDS|     262144920| 2015-08-31|
| RZKBT035JA0UQ|   16164990|B00X797LUS|     883589001| 2015-08-31|
|R253N5W74SM7N3|   33386989|B00C6MXB42|     734735137| 2015-08

In [ ]:
product_df=cleaned_df.select(["product_id", "product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B005T4ND06|Yoga for Movement...|
|B004EPZ070|  Something Borrowed|
|B005S9EKCW|Les Miserables (2...|
|B00FC1ZCB4|Alien Anthology a...|
|B002ZG98Z0|  Sex and the City 2|
|B00X8RONBO|When Calls The He...|
|B000CEXFZG|Teen Titans - The...|
|B00ID8H8EW|      Generation War|
|B000TGJ8IU|Troy  (Director's...|
|B00DPMPTDS|Faith Aka the Gre...|
|B00X797LUS|   Revenge: Season 4|
|B00C6MXB42|YOUNG INDIANA JON...|
|B000EZ9084|     Survival Island|
|B000CEXFZG|Teen Titans - The...|
|B00KXEM6XM|     Fargo: Season 1|
|B0036K9CM8|Disney Parks: The...|
|B00G3HOJZ6|Young Detective D...|
|B00V5E7YR2|The Divergent Ser...|
|B005BUA1JY|Barney Miller: Th...|
|B00YCY46VO|Inside Out (Blu-r...|
+----------+--------------------+
only showing top 20 rows



In [8]:
customer_df=cleaned_df.groupby("customer_id").count()
customer_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    9507671|   13|
|   15080145|    1|
|   25072529|    1|
|   31325270|    2|
|    9058434|    2|
|     220616|    3|
|   48428870|   25|
|   22848554|    8|
|    7763633|    1|
|   43482708|   12|
|   24139636|    1|
|    5884899|    7|
|   42200307|    1|
|   43963799|    1|
|   36879575|    1|
|   17345393|    6|
|   18460373|    3|
|   29351344|    1|
|   52809621|    2|
|   52857540|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
vinetable_df=cleaned_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vinetable_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R33UPQQUZQEM8|          5|            3|          3|   N|
|R3IKTNQQPD9662|          5|            0|          0|   N|
|R3U27V5QMCP27T|          5|            1|          1|   N|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|
|R3OM9S0TCBP38K|          5|            0|          0|   N|
|R1W4S949ZRCTBW|          5|            0|          0|   N|
|R18JL1NNQAZFV2|          5|            0|          0|   N|
|R1LP6PR06OPYUX|          4|            0|          0|   N|
| RZKBT035JA0UQ|          5|            1|          2|   N|
|R253N5W74SM7N3|          4|            1|          1|   N|
|R2D5IFTFPHD3RN|          4|            1|          1|   N|
|R1CECK3H1URK1G|          5|            

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:

parent_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:

product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:

customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:

vinetable_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)